## Set-up

In [1]:
import tifffile
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from pathlib import Path
import gc
from matplotlib import colormaps as cm

import sys
import os

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

import utils
import track

2024-03-28 08:46:47.089445: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-28 08:46:47.116262: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 08:46:47.116290: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 08:46:47.117108: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-28 08:46:47.121943: I tensorflow/core/platform/cpu_feature_guar

### Hyper parameters

In [2]:
loadstar_weight_name = "loadstar_first"
magik_weight_name = "MAGIK_MP_MPN"

# Experiment parameters
tiff_path = r'/home/gideon/Documents/PhD/Projects/KAW/data_from_VE/Experiment-16031.tif'
output_path = r'/home/gideon/Documents/PhD/Projects/KAW/output'
pages_batch_size = 2
channel_width = 0.3 # Microns

# Loadstar parameters
alpha = 0.999
cutoff = 1e-2

# MAGIK parameters
radius = 0.08 # Value between 0 and 1 where 1 is the full image hight
traj_min = 6
n_frames = 7

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

loadstar, magik = utils.load_models(loadstar_weight_name, magik_weight_name)

Num GPUs Available:  1


2024-03-28 08:46:49.741113: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-28 08:46:49.741302: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-28 08:46:49.741385: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

### Extract metadata

In [3]:
tiffpage = tifffile.TiffFile(tiff_path).pages[0]
for tag in tiffpage.tags.values():
    print(tag.name, tag.value)

NewSubfileType 0
ImageWidth 1254
ImageLength 1254
BitsPerSample 16
Compression 1
PhotometricInterpretation 1
ImageDescription ImageJ=1.54f
images=220
frames=220
unit=micron
finterval=0.45014611872146115
loop=false
min=254.0
max=255.0
StripOffsets (19051,)
SamplesPerPixel 1
RowsPerStrip 1254
StripByteCounts (3145032,)
XResolution (9090909, 1000000)
YResolution (9090909, 1000000)
ResolutionUnit 1
IJMetadataByteCounts (12, 78, 78, 78, 78, 78, 78, 78, 78, 78, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 

In [4]:
width = tiffpage.tags.get('ImageWidth').value # Pixels
height = tiffpage.tags.get('ImageLength').value # Pixels
x_resolution = tiffpage.tags.get('XResolution').value # Pixels per micron
y_resolution = tiffpage.tags.get('YResolution').value # Pixels per micron

### Calculated paramters

In [5]:
# Calculated parameters
x_resolution = x_resolution[0] / 1000000 # Pixels per micron
y_resolution = y_resolution[0] / 1000000 # Pixels per micron

pixel_size_x = 1/x_resolution # Microns per pixel
pixel_size_y = 1/y_resolution # Microns per pixel

channel_width_pixels = channel_width * x_resolution # Pixels
channel_width_norm = channel_width_pixels / width

radius_ratio = radius / channel_width_norm

print(f"Channel width: {channel_width_pixels} pixels")
print(f"Radius ratio: {radius_ratio} ratio")

Channel width: 2.7272727 pixels
Radius ratio: 36.78400036784 ratio


## Detect

In [12]:
detections  = utils.detect(tiff_path, loadstar, pages_batch_size, alpha, cutoff)
utils.save_detections(detections, "../../output/detections-2.csv", full=True)

100%|██████████| 110/110 [01:02<00:00,  1.75it/s]


2259
Predicting trajectories...
Creating graph edges...
Creating dataframe...
Getting trajectories...
Done!
Predicting trajectories...
Creating graph edges...
Creating dataframe...
Getting trajectories...
Done!


## Track

In [6]:
detections = utils.load_detections("../../output/detections-2.csv")

detections = utils.warp_axis(detections, radius_ratio, axis_name='centroid-0')
tracks = track.predict_trajectories(magik, detections, radius, n_frames, traj_min)
tracks = utils.unwarp_axis(tracks, radius_ratio, axis_name='x')

# Convert to pixel values
tracks['x'] *= width
tracks['y'] *= height
track.save_trajectories(tracks, "../../output/tracks.csv")

Predicting trajectories...
Creating graph edges...


100%|██████████| 1/1 [00:03<00:00,  3.53s/it]
2024-03-28 08:47:20.918986: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


Creating dataframe...
Getting trajectories...


/home/gideon/Documents/PhD/Projects/KAW/.venv/lib/python3.11/site-packages/deeptrack/models/gnns/utils.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  d.loc[
/home/gideon/Documents/PhD/Projects/KAW/.venv/lib/python3.11/site-packages/deeptrack/models/gnns/utils.py:52: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  alpha = 0.1 + d_alpha * int(solutions[i : i + 1]["frame_y"])
/home/gideon/Documents/PhD/Projects/KAW/.venv/lib/python3.11/site-packages/deeptrack/models/gnns/utils.py:54: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  if not (str(int(solutions[i : i + 1]["node_x"])) in t.keys()):
/home/gideon/Documents/PhD/Pro

Done!


### Plot track

In [18]:
track.plot_trajectories(tracks, tiff_path, output_path, pages_batch_size, stop=50)

100%|██████████| 26/26 [00:33<00:00,  1.30s/it]


## Further analyze the tracks

#### Add axis for lengths in microns

In [8]:
tracks = track.load_trajectories("../../output/tracks.csv")

tracks['x_microns'] = tracks['x'] * pixel_size_x
tracks['y_microns'] = tracks['y'] * pixel_size_y

#### Alt. replace x and y with micron versions

In [ ]:
tracks['x'] = tracks['x'] * pixel_size_x
tracks['y'] = tracks['y'] * pixel_size_y

In [9]:
tracks.head(20)

,frame,y,x,entity,x_microns,y_microns
0,0.0,102.987546,1238.463943,1.0,136.231035,11.328630
1,0.0,251.217419,436.453195,2.0,48.009852,27.633916
2,0.0,257.965005,213.490979,3.0,23.484008,28.376151
3,0.0,306.162532,437.811566,10.0,48.159273,33.677879
4,0.0,515.116151,606.502028,4.0,66.715224,56.662777
5,0.0,757.485722,458.801431,5.0,50.468158,83.323430
6,0.0,1025.031262,31.761879,6.0,3.493807,112.753440
7,0.0,1072.599746,1038.329180,7.0,114.216211,117.985973
8,1.0,104.138192,1238.217783,1.0,136.203957,11.455201
9,1.0,251.193142,435.768285,2.0,47.934512,27.631246


### Remove still particles

In [41]:
tracks = track.load_trajectories("../../output/tracks.csv")

still_cuttoff = 40

print(f'Number of objects before: {len(tracks.entity.unique())}')

tracks_stills_removed = track.remove_still_objects(tracks, still_cuttoff)

print(f'Number of objects after: {len(tracks_stills_removed.entity.unique())}')

In [ ]:
track.plot_trajectories(tracks_stills_removed, tiff_path, output_path, pages_batch_size, stop=50)

100%|██████████| 26/26 [00:34<00:00,  1.32s/it]


### Track length and distance

In [ ]:
tracks = tracks_stills_removed
tracks = track.distance_traveled(tracks)
tracks.head(20)

,frame,y,x,entity,delta_y,delta_x,traveled_y,traveled_x
3,0.0,303.690397,1107.441383,4.0,0.000000,0.000000,0.000000,0.000000
12,1.0,309.889496,1107.882415,4.0,6.199099,0.441032,6.199099,0.441032
28,3.0,363.002239,1106.338490,4.0,53.112744,1.543925,59.311842,1.984957
39,4.0,393.835002,1108.881728,4.0,30.832763,2.543237,90.144605,4.528194
52,5.0,415.304397,1107.900097,4.0,21.469395,0.981631,111.614001,5.509825
66,6.0,461.942024,1107.695444,4.0,46.637627,0.204653,158.251627,5.714478
81,7.0,474.217342,1107.783976,4.0,12.275318,0.088532,170.526946,5.803010
96,8.0,498.963365,1108.995089,4.0,24.746022,1.211113,195.272968,7.014124
112,9.0,507.723947,1106.912321,4.0,8.760582,2.082769,204.033550,9.096892
130,10.0,540.135846,1110.027006,4.0,32.411900,3.114685,236.445449,12.211577


In [ ]:
tracks = track.count_appearances(tracks)
track.save_trajectories(tracks, "../../output/tracks.csv")

tracks.head(50)


,frame,y,x,entity,delta_y,delta_x,traveled_y,traveled_x,frame_count
3,0.0,303.690397,1107.441383,4.0,0.000000,0.000000,0.000000,0.000000,0.0
12,1.0,309.889496,1107.882415,4.0,6.199099,0.441032,6.199099,0.441032,1.0
28,3.0,363.002239,1106.338490,4.0,53.112744,1.543925,59.311842,1.984957,3.0
39,4.0,393.835002,1108.881728,4.0,30.832763,2.543237,90.144605,4.528194,4.0
52,5.0,415.304397,1107.900097,4.0,21.469395,0.981631,111.614001,5.509825,5.0
66,6.0,461.942024,1107.695444,4.0,46.637627,0.204653,158.251627,5.714478,6.0
81,7.0,474.217342,1107.783976,4.0,12.275318,0.088532,170.526946,5.803010,7.0
96,8.0,498.963365,1108.995089,4.0,24.746022,1.211113,195.272968,7.014124,8.0
112,9.0,507.723947,1106.912321,4.0,8.760582,2.082769,204.033550,9.096892,9.0
130,10.0,540.135846,1110.027006,4.0,32.411900,3.114685,236.445449,12.211577,10.0
